In [1]:
import string
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.metrics import classification_report
import os
import urllib.request   
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM,Flatten
import numpy as np
import gensim
from gensim import corpora, models, similarities
from gensim.parsing.preprocessing import remove_stopwords    
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.
C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
if not os.path.exists('data'):
    os.mkdir('data')

urllib.request.urlretrieve('https://www.gutenberg.org/files/2591/2591-0.txt', 'data/Jacob and Wilhelm/grimms_fairy_tales.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/345/pg345.txt', 'data/Bram Stoker/dracula.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/84/pg84.txt', 'data/Mary Shelley/frankenstein.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/files/2701/2701-0.txt', 'data/Herman Melville/moby_dick.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/files/74/74-0.txt', 'data/Mark Twain/tom_sawyer.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/38189/pg38189.txt', 'data/Ambrose Blacklock/A_TREATISE_ON_SHEEP.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/2166/pg2166.txt', 'data/H.Rider Haggard/King_Solomons_Mines.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/21728/pg21728.txt', 'data/R.M.Ballantyne/THE_DOG_CRUSOE_AND_HIS_MASTER.txt')
urllib.request.urlretrieve('https://www.gutenberg.org/files/95/95-0.txt', 'data/Anthony Hope/THE_PRISONER_OF_ZENDA.txt')
urllib.request.urlretrieve('https://www.gutenberg.org/files/1257/1257-0.txt', 'data/Alexandre Dumas/THE_THREE_MUSKETEERS.txt')


('data/Alexandre Dumas/THE_THREE_MUSKETEERS.txt',
 <http.client.HTTPMessage at 0x18a7dc06a20>)

In [3]:
path = './data'
token_dict = {}
dfs=[]



for dirpath, dirs, files in os.walk(path):
    for f in files:
        fname = os.path.join(dirpath, f)
        fdir=os.path.basename(dirpath)
        print ("fname=", fname)
        dt=pd.read_fwf(fname, sep=".", header=None)
        dt['label']=fdir
        dfs.append(dt)

            
big_frame = pd.concat(dfs, ignore_index=True)      

fname= ./data\Alexandre Dumas\THE_THREE_MUSKETEERS.txt
fname= ./data\Ambrose Blacklock\A_TREATISE_ON_SHEEP.txt
fname= ./data\Anthony Hope\THE_PRISONER_OF_ZENDA.txt
fname= ./data\Bram Stoker\dracula.txt
fname= ./data\H.Rider Haggard\King_Solomons_Mines.txt
fname= ./data\Herman Melville\moby_dick.txt
fname= ./data\Jacob and Wilhelm\grimms_fairy_tales.txt
fname= ./data\Mark Twain\tom_sawyer.txt
fname= ./data\Mary Shelley\frankenstein.txt
fname= ./data\R.M.Ballantyne\THE_DOG_CRUSOE_AND_HIS_MASTER.txt


C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [4]:
big_frame.head()

,0,label,1
0,ï»¿ THE THREE MUSKETEERS,Alexandre Dumas,NaN
1,This ebook is for the use of anyone anywhere i...,Alexandre Dumas,NaN
2,most other parts of the world at no cost and w...,Alexandre Dumas,NaN
3,"whatsoever. You may copy it, give it away or r...",Alexandre Dumas,NaN
4,of the Project Gutenberg License included with...,Alexandre Dumas,NaN


In [5]:
big_frame["label"].value_counts()

Alexandre Dumas      22602
Herman Melville      19222
Bram Stoker          13414
Jacob and Wilhelm     9571
R.M.Ballantyne        7830
H.Rider Haggard       7451
Mark Twain            6927
Ambrose Blacklock     6577
Anthony Hope          5206
Mary Shelley          1300
Name: label, dtype: int64

In [6]:
big_frame.dtypes
big_frame[0] = big_frame[0].replace('"', ' ', regex=True)
big_frame[0] = big_frame[0].replace('!', ' ', regex=True)
big_frame[0] = big_frame[0].replace('-', ' ', regex=True)
big_frame[0] = big_frame[0].replace(',', ' ', regex=True)
big_frame[0] = big_frame[0].replace('_', ' ', regex=True)
big_frame[0] = big_frame[0].replace(';', ' ', regex=True)



In [7]:
df_x=big_frame[0].values.astype('U')
df_y=big_frame["label"]


In [8]:

stop_words = stopwords.words('english') + list(punctuation)
 
def tokenize(text):
    words = word_tokenize(text)
    words = [w.lower() for w in words]
    return [w for w in words if w not in stop_words and not w.isdigit()]



## 1/ Naive Bayes classifier 

In [9]:

cv = TfidfVectorizer(tokenizer=tokenize,min_df=1,ngram_range =(1,2))
x_trainc=cv.fit_transform(df_x)
x_train, x_test, y_train, y_test = train_test_split(x_trainc, df_y, test_size=0.1, random_state=4)
mnb = MultinomialNB(alpha=0.1)
clf=mnb.fit(x_train,y_train)


In [10]:
predicted = clf.predict(x_test)

In [11]:
print('%.1f%%' % (np.mean(predicted == y_test) * 100))

70.5%


In [12]:
print(classification_report(y_test, predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.68      0.88      0.77      2266
Ambrose Blacklock       0.75      0.67      0.71       638
     Anthony Hope       0.87      0.31      0.46       515
      Bram Stoker       0.58      0.71      0.64      1348
  H.Rider Haggard       0.74      0.43      0.54       778
  Herman Melville       0.71      0.81      0.76      1907
Jacob and Wilhelm       0.81      0.78      0.79       950
       Mark Twain       0.85      0.48      0.62       665
     Mary Shelley       1.00      0.84      0.92       135
   R.M.Ballantyne       0.76      0.58      0.66       808

      avg / total       0.72      0.71      0.70     10010



## 2/ Stochastic Gradient Descent classifier

In [13]:
svm=SGDClassifier()
svm_classifier=svm.fit(x_train,y_train)

C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [14]:
svm_predicted = svm_classifier.predict(x_test)
print('%.1f%%' % (np.mean(svm_predicted == y_test) * 100))

68.2%


In [15]:
print(classification_report(y_test, svm_predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.68      0.86      0.76      2266
Ambrose Blacklock       0.67      0.72      0.69       638
     Anthony Hope       0.70      0.33      0.45       515
      Bram Stoker       0.66      0.64      0.65      1348
  H.Rider Haggard       0.71      0.40      0.51       778
  Herman Melville       0.65      0.78      0.71      1907
Jacob and Wilhelm       0.72      0.75      0.74       950
       Mark Twain       0.70      0.47      0.56       665
     Mary Shelley       0.99      0.84      0.91       135
   R.M.Ballantyne       0.72      0.55      0.62       808

      avg / total       0.69      0.68      0.67     10010



In [16]:
big_frame = pd.get_dummies(big_frame, columns=['label'])

In [18]:
big_frame

,0,1,label_Alexandre Dumas,label_Ambrose Blacklock,label_Anthony Hope,label_Bram Stoker,label_H.Rider Haggard,label_Herman Melville,label_Jacob and Wilhelm,label_Mark Twain,label_Mary Shelley,label_R.M.Ballantyne
0,ï»¿ THE THREE MUSKETEERS,NaN,1,0,0,0,0,0,0,0,0,0
1,This ebook is for the use of anyone anywhere i...,NaN,1,0,0,0,0,0,0,0,0,0
2,most other parts of the world at no cost and w...,NaN,1,0,0,0,0,0,0,0,0,0
3,whatsoever. You may copy it give it away or r...,NaN,1,0,0,0,0,0,0,0,0,0
4,of the Project Gutenberg License included with...,NaN,1,0,0,0,0,0,0,0,0,0
5,http://www.gutenberg.org/license. If you are n...,NaN,1,0,0,0,0,0,0,0,0,0
6,States youâ€™ll have to check the laws of the...,NaN,1,0,0,0,0,0,0,0,0,0
7,located before using this ebook.,NaN,1,0,0,0,0,0,0,0,0,0
8,Title: The Three Musketeers,NaN,1,0,0,0,0,0,0,0,0,0
9,Author: Alexandre Dumas Pere,NaN,1,0,0,0,0,0,0,0,0,0


In [ ]:
big_frame.columns

In [20]:
feature_cols=['label_Alexandre Dumas', 'label_Ambrose Blacklock',
            'label_Anthony Hope',       'label_Bram Stoker',
         'label_H.Rider Haggard',   'label_Herman Melville',
       'label_Jacob and Wilhelm',        'label_Mark Twain',
            'label_Mary Shelley',    'label_R.M.Ballantyne']
y=big_frame[feature_cols]
big_frame.to_csv('big_frame.csv', encoding='utf-8')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_trainc, y, test_size=0.1, random_state=4)
x_trainc.shape

In [ ]:
top_words = 50000
max_review_length = 423107

embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(32))
model.add(Dense(10, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=3, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))